##### Basic configuration

In [ ]:
bs = 12
saved_model = './saved_model/unet'

##### Load data

In [ ]:
import data_io

trainset_loader = data_io.ImageDataGenerator2(batch_size=bs)
validset_loader = data_io.ImageDataGenerator2(batch_size=bs, mode='valid')

##### Train
1. **Compile the network**: using the BinaryCrossentropy as loss function and the Adam as optimizor.
2. **Set 2 callbacks**: to automatically reduce learning rate when the val_loss is not improved in a period of time and to stop the trainning if reduce lr cannot improve the performance.

In [ ]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow.keras.optimizers as optim
import network
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

model = network.cnn_dense()
model.summary()
model.compile(loss=BinaryCrossentropy(from_logits=False), optimizer=optim.Adam())


reduce_lr = ReduceLROnPlateau(
    verbose=1, factor=0.1, min_delta=0.1, monitor='val_loss', patience=10,
    mode='auto', min_lr=0.00001)
stop_condition = EarlyStopping(
    monitor='val_loss', patience=21, min_delta=0.1)

history = model.fit(trainset_loader, batch_size=bs, epochs=100,
                    validation_data=validset_loader, workers=16, max_queue_size=16,
                    callbacks=[reduce_lr, stop_condition])

model.save(saved_model)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('./figure/UNet - Model loss6.jpg')